# Anno 1800: Maximize population by adjusting skyscraper levels
![Run all cells](imgs/run_all.png) ![Restart kernel](imgs/restart_kernel.png)

In [ ]:
import ipywidgets as widgets
with open("imgs/loading-buffering.gif", "rb") as f:
    spinner = widgets.Image(
        value=f.read(),
        format='gif',
        width="5%",
        margin="auto"
    )
    spinner.layout.margin="0 0 0 47.5%"
    display(spinner)

In [ ]:
import importlib
from IPython.display import clear_output

if importlib.util.find_spec("pulp") is None :
    ! pip install --user pulp
    clear_output(wait=True) 
    
from pulp import *

available_solvers = listSolvers(onlyAvailable=True)
clear_output(wait=True) 

if "GUROBI_CMD" in available_solvers :
    if importlib.util.find_spec("gurobipy") is None :
        ! pip install --user gurobipy
        clear_output(wait=True)

from tools.skyscraper_levels import *
    
try:
    spinner.close()
except:
    pass

clear_output(wait=True) 
ui = OptimizerGUI()

In [ ]:
if hasattr(ui, "logs"):
    df = pd.DataFrame(ui.logs, columns=["Time", "Best Found", "Upper Bound"])
    display(df.plot(kind="scatter", x=["Time","Time"], y=["Best Found", "Upper Bound"], s=5,
                    #remove '#' in front of ylim and change limits to zoom in
                    #ylim=(600000,650000),
                    #xlim=(0,600)
                   ))